In [12]:
!pip install -U gdown
!gdown "https://drive.google.com/uc?id=15Z_F4Dwgb3NLqEGnVMUEJqyxXgW7Gx-h"

Downloading...
From (original): https://drive.google.com/uc?id=15Z_F4Dwgb3NLqEGnVMUEJqyxXgW7Gx-h
From (redirected): https://drive.google.com/uc?id=15Z_F4Dwgb3NLqEGnVMUEJqyxXgW7Gx-h&confirm=t&uuid=407e3b11-5983-4f4a-ac3a-6fa0d7cdafb3
To: /content/blobs_crawled_data.zip
100% 12.5M/12.5M [00:00<00:00, 18.5MB/s]


In [13]:
!unzip blobs_crawled_data.zip

Archive:  blobs_crawled_data.zip
   creating: blobs_crawled_data/
  inflating: blobs_crawled_data/metadata.csv  
   creating: blobs_crawled_data/images/
 extracting: blobs_crawled_data/images/Blobs_0002712.jpg  
  inflating: blobs_crawled_data/images/Blobs_0003414.jpg  
 extracting: blobs_crawled_data/images/Blobs_0006860.jpg  
 extracting: blobs_crawled_data/images/Blobs_0000636.jpg  
 extracting: blobs_crawled_data/images/Blobs_0001219.jpg  
 extracting: blobs_crawled_data/images/Blobs_0005969.jpg  
 extracting: blobs_crawled_data/images/Blobs_0007068.jpg  
 extracting: blobs_crawled_data/images/Blobs_0000014.jpg  
 extracting: blobs_crawled_data/images/Blobs_0003213.jpg  
 extracting: blobs_crawled_data/images/Blobs_0005203.jpg  
 extracting: blobs_crawled_data/images/Blobs_0003097.jpg  
 extracting: blobs_crawled_data/images/Blobs_0018822.jpg  
 extracting: blobs_crawled_data/images/Blobs_0004159.jpg  
 extracting: blobs_crawled_data/images/Blobs_0000326.jpg  
 extracting: blobs_cr

In [6]:
pip install qwen-vl-utils

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.3/35.3 MB 23.2 MB/s eta 0:00:00


In [14]:
import os
import pandas as pd        # Xử lý dữ liệu bảng
import torch               # PyTorch - deep learning
from transformers import Qwen2_5_VLForConditionalGeneration
from transformers import AutoProcessor
from PIL import Image      # Xử lý ảnh
from qwen_vl_utils import process_vision_info
from tqdm import tqdm      # Hiển thị tiến độ

In [8]:
model_id = "Qwen/Qwen2.5-VL-3B-Instruct"

model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

processor = AutoProcessor.from_pretrained(model_id)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.37k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/65.4k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.98G [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

You have video processor config saved in `preprocessor.json` file which is deprecated. Video processor configs should be saved in their own `video_preprocessor.json` file. You can rename the file or load and save the processor back which renames it automatically. Loading from `preprocessor.json` will be removed in v5.0.


chat_template.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

In [15]:
def create_prompt(emoji_title):
    instruction = (
        "Instruction: Given the emoji title and the corresponding image, generate a natural language description "
        "of the emoji's key features. The description should include: 1) the head shape, 2) eye characteristics, "
        "3) mouth characteristics, 4) facial expression, 5) skin color, 6) any action (if present), and 7) background color. "
        "The description should be concise and structured like this: "
        "'{emoji_title} emoji with a {head shape}, {eye description} eyes, {mouth description} mouth, "
        "{expression description} expression, {skin color} skin color, {action description}, "
        "with a {background color} background.'"
    )

    example = (
        "Example: For an emoji titled ‘Pepe the Frog’, the description should be like: "
        "'Pepe the frog emoji with a round head, big eyes, smiling mouth, happy expression, "
        "green skin color, no action, with a green background.'"
    )

    prompt = f"{instruction}\n{example}\nInput: '{emoji_title}'"
    return prompt


In [16]:
def process_subfolders(root_dir, data_folder):
    for subfolder in data_folder:
        subfolder_path = os.path.join(root_dir, subfolder)

        if os.path.isdir(subfolder_path):
            images_folder = os.path.join(subfolder_path, "images")
            metadata_file = os.path.join(subfolder_path, "metadata.csv")

            if os.path.exists(images_folder) and os.path.exists(metadata_file):
                df = pd.read_csv(metadata_file)
                df["prompt"] = ""

                for _, row in tqdm(df.iterrows(), total=len(df), desc=f"Processing {subfolder}"):
                    image_path = os.path.join(images_folder, row["file_name"])
                    prompt = create_prompt(row["image_title"])

                    image = Image.open(image_path).convert("RGBA")

                    messages = [
                        {
                            "role": "user",
                            "content": [
                                {"type": "image", "image": image},
                                {"type": "text", "text": prompt},
                            ],
                        },
                    ]

                    text = processor.apply_chat_template(
                        messages, tokenize=False, add_generation_prompt=True
                    )
                    image_inputs, video_inputs = process_vision_info(messages)

                    inputs = processor(
                        text=[text],
                        images=image_inputs,
                        videos=video_inputs,
                        padding=True,
                        return_tensors="pt"
                    ).to("cuda")

                    generated_ids = model.generate(**inputs, max_new_tokens=128)
                    generated_ids_trimmed = [
                        out_ids[len(in_ids):] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
                    ]
                    result_text = processor.batch_decode(
                        generated_ids_trimmed,
                        skip_special_tokens=True,
                        clean_up_tokenization_spaces=False
                    )[0]

                    df.loc[_, "prompt"] = result_text

                df.to_csv(metadata_file, index=False)


In [ ]:
data_folder = ["blobs_crawled_data"]
root_dir = "./"
process_subfolders(root_dir, data_folder)

Processing blobs_crawled_data:   0%|          | 8/4621 [00:24<3:22:12,  2.63s/it]